## Notebook for training a BERT-BCA model

This notebook can be used to train a BCA model with BERT token embedding as input. The input can be generated using BERT as service (https://github.com/hanxiao/bert-as-service). The script BERT_text_representation.py can be used to create the BERT embedding from the TOEFL essay data after running TOEFL_dataParse.py 

In [ ]:
from __future__ import print_function, division

import os
import os.path
import pandas as pd
from io import StringIO
import io
import unicodedata
import re
import random

import tensorflow as tf
import numpy as np
np.set_printoptions(threshold = 10000)
import collections
import random

from tensorflow.contrib.rnn import LSTMCell as Cell #for GRU: custom implementation with normalization
from tensorflow.python.ops.rnn import dynamic_rnn as rnn
from tensorflow.python.ops.rnn import bidirectional_dynamic_rnn as bi_rnn
from tensorflow.contrib.rnn import DropoutWrapper

from attention import attention as attention
from bca_ import *
from ordloss import *
from utils import *
#from datautilsbca import *


from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from scipy import stats
from sklearn.metrics import accuracy_score
 

In [ ]:
SEQUENCE_LENGTH = 40
SEQUENCE_LENGTH_D = 25
max_vocab = 75000
train_split = 0.9
BATCH_SIZE = 20

# system parameters
HIDDEN_SIZE = 150
HIDDEN_SIZE_D = 150
ATTENTION_SIZE = 75
ATTENTION_SIZE_D = 50
LAYER_1 = 500
LAYER_2 = 250
LAYER_3 = 100
KEEP_PROB = 0.7
#NUM_EPOCHS = 1  # max val_acc at __
DELTA = 0.75

In [ ]:
X_train = np.load('data/TOEFL/X_train_TOEFL.npy')
y_train = np.load('data/TOEFL/y_train_TOEFL.npy')

In [ ]:
SEQUENCE_LEN_D = SEQUENCE_LENGTH_D
SEQUENCE_LEN = SEQUENCE_LENGTH
tr_len = int(train_split*len(y_train))
X_train, y_train, X_val, y_val  = X_train[:tr_len*SEQUENCE_LEN_D], y_train[:tr_len], X_train[tr_len*SEQUENCE_LEN_D:], y_train[tr_len:]

In [ ]:
tf.reset_default_graph()
tf.set_random_seed(111)
b_len = 768

In [ ]:
#Different placeholders
#num_classes = y_train.shape[1]
num_classes = 3
batch_ph = tf.placeholder(tf.float32, [None, SEQUENCE_LENGTH, b_len])
ind_list_ph = tf.placeholder(tf.int32, [None])
target_ph = tf.placeholder(tf.float32, [None,num_classes])

seq_len_ph = tf.placeholder(tf.int32, [None])
seq_len_ph_d = tf.placeholder(tf.int32, [None])
keep_prob_ph = tf.placeholder(tf.float32)
doc_size_ph = tf.placeholder(tf.int32,[None])


In [ ]:
W_omega = tf.Variable(tf.random_uniform([HIDDEN_SIZE*2, HIDDEN_SIZE*2], -1.0, 1.0))

# (Bi-)RNN layer(-s)
with tf.variable_scope('sentence'):
    fw_cell = Cell(HIDDEN_SIZE)
    bw_cell = Cell(HIDDEN_SIZE)
    
    fw_cell = DropoutWrapper(fw_cell, input_keep_prob=keep_prob_ph, 
                             output_keep_prob=keep_prob_ph,state_keep_prob=keep_prob_ph,
                             variational_recurrent=True, input_size=batch_ph.get_shape()[-1], 
                             dtype = tf.float32)
    bw_cell = DropoutWrapper(bw_cell, input_keep_prob=keep_prob_ph, 
                             output_keep_prob=keep_prob_ph,state_keep_prob= keep_prob_ph,
                             variational_recurrent=True, input_size=batch_ph.get_shape()[-1], 
                             dtype = tf.float32)
    rnn_output, _ = bi_rnn(fw_cell, bw_cell, inputs=batch_ph, sequence_length=seq_len_ph, dtype=tf.float32)

    
    rnn_outputs_ = cross_attention(rnn_output, SEQUENCE_LENGTH_D, seq_len_ph, BATCH_SIZE, W_omega)
    attention_output_, alphas_ = attention(rnn_outputs_ , ATTENTION_SIZE, seq_len_ph, return_alphas = True)
    attention_output_ = tf.reshape(attention_output_,[BATCH_SIZE, -1, HIDDEN_SIZE*2*3])
    
with tf.variable_scope('document'):
    fw_cell_d = Cell(HIDDEN_SIZE_D)
    bw_cell_d = Cell(HIDDEN_SIZE_D)
    
    fw_cell_d = DropoutWrapper(fw_cell_d, input_keep_prob=keep_prob_ph, 
                             output_keep_prob=keep_prob_ph,state_keep_prob=keep_prob_ph,
                             variational_recurrent=True, input_size=attention_output_.get_shape()[-1], 
                             dtype = tf.float32)
    bw_cell_d = DropoutWrapper(bw_cell_d, input_keep_prob=keep_prob_ph, 
                             output_keep_prob=keep_prob_ph,state_keep_prob= keep_prob_ph,
                             variational_recurrent=True, input_size=attention_output_.get_shape()[-1], 
                             dtype = tf.float32)
    rnn_outputs_d, _ = bi_rnn(fw_cell_d, bw_cell_d, inputs=attention_output_, 
                              sequence_length=seq_len_ph_d, dtype=tf.float32)
    
    attention_output_d, alphas_d = attention(rnn_outputs_d, ATTENTION_SIZE_D, seq_len_ph_d, return_alphas=True)

# Dropout
drop = tf.nn.dropout(attention_output_d, keep_prob_ph)

In [ ]:
ordinal = True
if ordinal:
    # For ordinal regression, same weights for each class
    W = tf.Variable(tf.truncated_normal([drop.get_shape()[1].value], stddev=0.1))
    W_ = tf.transpose(tf.reshape(tf.tile(W,[num_classes - 1]),[num_classes - 1, drop.get_shape()[1].value]))
    b = tf.Variable(tf.cast(tf.range(num_classes - 1), dtype = tf.float32))
    y_hat_ = tf.nn.xw_plus_b(drop, tf.negative(W_), b)

    # Predicted labels and logits
    y_preds, logits = preds(y_hat_,BATCH_SIZE)
    y_true = tf.argmax(target_ph, axis = 1)

    # Ordinal loss
    loss = ordloss_m(y_hat_, target_ph, BATCH_SIZE)
    c = stats.spearmanr
    str_score = "Spearman rank:"
    
# Calculate and clip gradients
max_gradient_norm = 5
lr = 1e-4
params = tf.trainable_variables()
gradients = tf.gradients(loss, params)
clipped_gradients, _ = tf.clip_by_global_norm(gradients, max_gradient_norm)
optimizer_ = tf.train.AdamOptimizer(learning_rate=lr)
optimizer = optimizer_.apply_gradients(
    zip(clipped_gradients, params))

In [ ]:
saver = tf.train.Saver()

In [ ]:
batch_counter = 0
val_counter = []
config = tf.ConfigProto(inter_op_parallelism_threads=24,
                        intra_op_parallelism_threads=24)
config.gpu_options.allow_growth = True
sess = tf.Session(config = config)
sess.run(tf.global_variables_initializer())

In [ ]:
#Main training task
train_batch_generator = batch_generator(X_train, y_train, BATCH_SIZE, seq_len = SEQUENCE_LENGTH_D)
val_batch_generator = batch_generator(X_val, y_val, BATCH_SIZE, seq_len = SEQUENCE_LENGTH_D)

train_accuracy = []
val_accuracy = []
val_counter = []
val_count = 50
NUM_EPOCHS = 60
doc_size_np = np.array([0]*SEQUENCE_LENGTH_D)
batch_counter = 0

loss_train = 0
sq_l = np.array([SEQUENCE_LENGTH]*BATCH_SIZE*SEQUENCE_LENGTH_D)
seq_len_d = [SEQUENCE_LENGTH_D]*BATCH_SIZE               
seq_len_d = np.array(seq_len_d)

print('Training on TOEFL data')
for epoch in range(NUM_EPOCHS):
    print("epoch: {}\t".format(epoch), end="")

    # Training
    num_batches = X_train.shape[0] // (BATCH_SIZE*SEQUENCE_LENGTH_D)
    true = []
    ypreds = []

    for bx in range(num_batches):
        batch_counter += 1
        x_batch, y_batch = next(train_batch_generator)
        
        y_preds_, loss_tr,  _  = sess.run([y_preds, loss,  optimizer],
                                   feed_dict={batch_ph: x_batch,
                                              target_ph: y_batch,
                                              seq_len_ph: sq_l,
                                              seq_len_ph_d: seq_len_d,
                                              doc_size_ph: doc_size_np,
                                              keep_prob_ph: KEEP_PROB})
        loss_train = loss_tr * DELTA + loss_train * (1 - DELTA)
        ypreds.extend(y_preds_)
        t = np.argmax(y_batch, axis = 1)
        true.extend(t)

        sp = c(y_preds_,t)
        if ordinal: 
            sp = sp[0]
        train_accuracy.append(sp)
        
        
        #testing on the validation set            
        if batch_counter%val_count == 0:
            val_counter.append(batch_counter)
            x_batch, y_batch = next(val_batch_generator)
            
            y_preds_,loss_t = sess.run([y_preds,loss],
                          feed_dict={batch_ph: x_batch,
                                target_ph: y_batch,
                                seq_len_ph: sq_l,
                                seq_len_ph_d: seq_len_d,
                                doc_size_ph: doc_size_np,
                                keep_prob_ph: 1.0})
            ypreds.extend(y_preds_)
            t = np.argmax(y_batch, axis = 1)
            true.extend(t)

            sp = c(y_preds_,t)
            if ordinal: 
                sp = sp[0]
            val_accuracy.append(sp)
            #saver.save(sess, MODEL_PATH, global_step = batch_counter)

    print('training loss: ' + str(loss_train))
    spr = c(true, ypreds)
    if ordinal:
        spr = spr[0]
    print('Training '+ str_score + str(spr))
    print('Val ' + str(np.mean(val_accuracy)))

In [ ]:
def zero_pad_test_(X, seq_len_div,b_len = 768):
    if (len(X)%seq_len_div) == 0:
        return np.array([x for x in X])
    diff = seq_len_div - (len(X)%seq_len_div)
    return np.concatenate((np.array([x for x in X]),np.zeros((diff,len(X[0]),b_len))), axis = 0)

def zero_pad_test(X, seq_len_div,b_len = 768):
    if (len(X)%seq_len_div) == 0:
        return np.array([x for x in X])
    diff = seq_len_div - (len(X)%seq_len_div)
    return np.concatenate((np.array([x for x in X]),np.zeros((diff,len(X[0])))), axis = 0)

In [ ]:
X_test = np.load('data/TOEFL/X_test_TOEFL.npy')
y_test = np.load('data/TOEFL/y_test_TOEFL.npy')
X_test = zero_pad_test_(X_test, BATCH_SIZE*SEQUENCE_LENGTH_D)
y_test = zero_pad_test(y_test, BATCH_SIZE)

In [ ]:
test_batch_generator = batch_generator(X_test, y_test, BATCH_SIZE, seq_len = SEQUENCE_LENGTH_D, shuffle = False)
#testing on the test set
num_batches = X_test.shape[0] // (BATCH_SIZE*SEQUENCE_LENGTH_D)
true = []
ypreds = []

for bx in range(num_batches):
    x_batch, y_batch = next(test_batch_generator)
    y_preds_= sess.run([y_preds],
                  feed_dict={batch_ph: x_batch,
                        target_ph: y_batch,
                        seq_len_ph: sq_l,
                        seq_len_ph_d: seq_len_d,
                        doc_size_ph: doc_size_np,
                        keep_prob_ph: 1.0})
    ypreds.extend(y_preds_)
    t = np.argmax(y_batch, axis = 1)
    true.extend(t)

In [ ]:
#len(ypreds[0])
ypreds = [j for sub in ypreds for j in sub]

In [ ]:
y_test_len = len(y_test)
true = true[:y_test_len]
ypreds = ypreds[:y_test_len]

spr = c(true, ypreds)

if ordinal:
    spr = spr[0]
print('Test set '+ str_score + str(spr))

rank = stats.spearmanr
print('sp rho')
print(rank(true, ypreds))

from sklearn.metrics import cohen_kappa_score as kappa
print('qwk')
print(kappa(true, ypreds, weights="quadratic"))

from scipy.stats import pearsonr
print('pearson')
print(pearsonr(true,ypreds))

print('kappa')
print(kappa(true, ypreds, weights=None))